In [17]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 15 14:49:58 2022

@author: Sidi Wu and Cédric Beaulac

Functional autoencoder implementation
"""

# Import modules
import sys
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
# import skfda as fda
# from skfda import representation as representation
# from skfda.exploratory.visualization import FPCAPlot
# # from skfda.exploratory.visualization import FPCAPlot
# # from skfda.preprocessing.dim_reduction import FPCA
# # from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
import sklearn
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev
import skfda

import skfda
from skfda import FDataGrid as fd
from skfda.representation.basis import BSpline as B


In [18]:
##Importation des données sous forme FDA

ElNino_OISST_region_1and2=skfda.datasets.fetch_cran("ElNino_OISST_region_1and2","rainbow")
ElNino_OISST_region_3=skfda.datasets.fetch_cran("ElNino_OISST_region_3","rainbow")
ElNino_OISST_region_4=skfda.datasets.fetch_cran("ElNino_OISST_region_4","rainbow")
ElNino_OISST_region_3and4=skfda.datasets.fetch_cran("ElNino_OISST_region_3and4","rainbow")


c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "sfts". The constructor for class "fts" will be used instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "fts". The constructor for class "fds" will be used instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "fds". The underlying R object is returned instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "sfts". The constructor for class "fts" will be used instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "fts". The constructor for class "fds" w

In [19]:
ElNino_OISST_region_1and2_y=ElNino_OISST_region_1and2['ElNino_OISST_region_1and2']['y'].to_numpy()
ElNino_OISST_region_1and2_x=ElNino_OISST_region_1and2['ElNino_OISST_region_1and2']['x']
# plt.plot(ElNino_OISST_region_1and2_x,ElNino_OISST_region_1and2_y)
# plt.show(
# )
ElNino_OISST_region_3_y=ElNino_OISST_region_3['ElNino_OISST_region_3']['y'].to_numpy()
ElNino_OISST_region_3_x=ElNino_OISST_region_3['ElNino_OISST_region_3']['x']
# plt.plot(ElNino_OISST_region_3_x,ElNino_OISST_region_3_y)
# plt.show(
# )

ElNino_OISST_region_3and4_y=ElNino_OISST_region_3and4['ElNino_OISST_region_3and4']['y'].to_numpy()
ElNino_OISST_region_3and4_x=ElNino_OISST_region_3and4['ElNino_OISST_region_3and4']['x']
# plt.plot(ElNino_OISST_region_3and4_x,ElNino_OISST_region_3and4_y)
# plt.show(
# )


ElNino_OISST_region_4_y=ElNino_OISST_region_4['ElNino_OISST_region_4']['y'].to_numpy()
ElNino_OISST_region_4_x=ElNino_OISST_region_4['ElNino_OISST_region_4']['x']
# plt.plot(ElNino_OISST_region_4_x,ElNino_OISST_region_4_y)
# plt.show(
# )

In [20]:
grid_points=np.arange(12)


In [21]:
x0=torch.tensor(ElNino_OISST_region_1and2_y).float().cuda()
x1=torch.tensor(ElNino_OISST_region_3_y).float().cuda()
x2=torch.tensor(ElNino_OISST_region_4_y).float().cuda()
x3=torch.tensor(ElNino_OISST_region_3and4_y).float().cuda()
x0=torch.t(x0)
x1=torch.t(x1)
x2=torch.t(x2)
x3=torch.t(x3)

In [22]:

labels = torch.tensor([0] * x0.shape[0] + [1] * x1.shape[0] + [2] * x2.shape[0] + [3] * x3.shape[0])
X=torch.concatenate([x0,x1,x2,x3])
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(X,labels,shuffle=True)
x_train=x_train.reshape(111,1,12).cuda()
y_train=y_train.unsqueeze(1).cuda()
x_test=x_test.reshape(37,1,12).cuda()
y_test=y_test.unsqueeze(1).cuda()

In [23]:
basis=B(knots=linspace(1,12,4),order=4)
basis.evaluate(eval_points=np.arange(12)).shape,fd(x_train[:,0,:].cpu()).to_basis(basis).coefficients.shape

basis_eval = basis.evaluate(np.arange(12))
basis_fc = torch.from_numpy(basis_eval[:, :, 0]).float()

basis_fc.shape

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_17632\1189248831.py:2: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis.evaluate(eval_points=np.arange(12)).shape,fd(x_train[:,0,:].cpu()).to_basis(basis).coefficients.shape
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_17632\1189248831.py:4: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_eval = basis.evaluate(np.arange(12))


torch.Size([6, 12])

In [24]:

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 3)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 3)
        torch.nn.init.constant_(m.bias.data, 0.0)


In [435]:
class Project_Classifier(nn.Module):
    def __init__(self):
        super(Project_Classifier,self).__init__()
    
        self.flatten=nn.Sequential(
            nn.Flatten(start_dim=1,end_dim=-1),

        )
        self.fc_block1=nn.Sequential(
        
        nn.Linear(6,256),
        nn.BatchNorm1d(256),
        nn.LeakyReLU(0.2),
        )
        self.fc_block2=nn.Sequential(
        nn.Linear(256,512),
        nn.BatchNorm1d(512),
        nn.LeakyReLU(0.2),
    )
        self.fc_block3=nn.Sequential(
        nn.Linear(512,32),
        nn.BatchNorm1d(32),
        nn.LeakyReLU(0.2),
        nn.Linear(32,4)
        )
    def Project(self,x,basis_fc):
        f=torch.matmul(x,basis_fc)
        return f
    
    
    def forward(self,x):
        grid=fd(x[:,0,:].cpu(),grid_points=grid_points)
        
        features=self.Project(x[:,0,:],basis_fc=basis_fc)
        flatten_out=self.flatten(features)
        out1=self.fc_block1(flatten_out)
        out2=self.fc_block2(out1)
        out3=self.fc_block3(out2)
        return out3.float().unsqueeze_(2).unsqueeze(3)








In [436]:
basis_fc=torch.t(torch.from_numpy(basis.evaluate(grid_points))[:,:,0]).float().cuda()
project_Classifier=Project_Classifier().cuda().apply(weights_init_normal)


C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_17632\43120078.py:1: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_fc=torch.t(torch.from_numpy(basis.evaluate(grid_points))[:,:,0]).float().cuda()


In [476]:
lr_adam=0.0001
lr_sgd=0.001
batch_size=32
betas=[0.45,0.98]
optimizer=optim.Adam(project_Classifier.parameters(),lr=lr_adam,betas=betas)
# optimizer=optim.SGD(project_Classifier.parameters(),lr=lr_sgd)
loss_function=nn.CrossEntropyLoss()


In [477]:

from tqdm import tqdm
def train(n_epochs):
    train_loss=torch.tensor(0).cuda().long()
    for epoch in tqdm(range(n_epochs)):
        print("Précision moyenne =",((torch.sum(torch.argmax(project_Classifier(x_test),dim=1).squeeze(1)==y_test)/37)*100).detach().cpu().numpy(),"%")  
        for i in range(int(len(x_train)/batch_size)):
            functions_train=x_train[batch_size*i:batch_size*(1+i),:,:]
            labels_train=y_train[batch_size*i:batch_size*(1+i)]
            optimizer.zero_grad()
            output=project_Classifier(functions_train)
            loss=loss_function(input=output,target=labels_train.unsqueeze(1).long())

            loss.backward()
            optimizer.step()
            train_loss+=loss.long()
        return train_loss,loss 

            


        

In [485]:
train(1000)
print("Précision moyenne =",((torch.sum(torch.argmax(project_Classifier(x_test),dim=1).squeeze(1)==y_test)/37)*100).detach().cpu().numpy(),"%")

  0%|          | 0/1000 [00:00<?, ?it/s]

Précision moyenne = 78.37838 %
Précision moyenne = 78.37838 %


In [247]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

65